In [2]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import math
import json
import torch.nn.functional as F
import torch
from torch import nn as nn
from tqdm.auto import tqdm
import youtokentome as yttm
import time
import random
import pickle


In [3]:
from nn_code.transformer_classes import *

In [6]:
torch

<module 'torch' from '/Users/t.zhordaniya/paraphrase/venv/lib/python3.7/site-packages/torch/__init__.py'>

In [7]:
from nn_code.bucketing import SequenceBucketingData

In [9]:
from nn_code.decode import greedy, sampling

In [10]:
tokenizer_path = 'models/saved_models/big_model/bpe_big.model'

In [11]:
tokenizer = yttm.BPE(model=tokenizer_path)

In [12]:
validation_loader_path = 'models/saved_models/big_model/valid_dataset_big.pickle'

In [13]:
with open(validation_loader_path, 'rb') as f:
    loader = pickle.load(f)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
INPUT_DIM = 16000
OUTPUT_DIM = 16000
HID_DIM = 256
ENC_LAYERS = 4
DEC_LAYERS = 4
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

In [16]:
model = Seq2Seq(enc, dec, 0, 0, device).to(device)

In [17]:
model_path = 'models/saved_models/big_model/big-model.pt'

In [18]:
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu') ))
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (tok_embedding): Embedding(16000, 256)
    (pos_embedding): Embedding(100, 256)
    (layers): ModuleList(
      (0): EncoderLayer(
        (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadAttentionLayer(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (positionwise_feedforward): PositionwiseFeedforwardLayer(
          (fc_1): Linear(in_features=256, out_features=512, bias=True)
          (fc_2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): EncoderLayer(
        (la

In [20]:
greedy(model, tokenizer, 'Что вы хотели спросить у меня', device)

'Что вы хотите чтобы я сказал'

In [43]:
sampling(model, tokenizer, 'Делай с этим что хочешь', device, temperature=1.5, top_k=10, top_p=0.8)

'С что ты хочешь чтобы ты хочешь дел с ак тебе с Дел что с Дел ай'